In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(override=True)

# Retrieve API keys from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY ")
PINECONE_ENV = os.getenv("PINECONE_ENV")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

Interesting link about poetry:  
https://www.yippeecode.com/topics/update-packages-in-python-poetry/  
   
Summary of commands discussed.  
  
poetry show -l  
poetry show -t  
poetry add "package==version"    
poetry add "package[extra]==version"  
poetry add "package==version" --dev  
poetry update  
poetry env list --full-path  
poetry env remove name-of-env  
poetry env use $(which python3)  
source $(dirname $(poetry run which python3))/activate  

In [8]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [9]:
output = llm('explain quantum mechanic in one sentence')
print(output)



Quantum mechanics is a physical theory that describes the behavior of matter and energy at the atomic and subatomic level.


In [11]:
print(llm.get_num_tokens('explain quantum mechanic in one sentence'))

7


In [12]:
output=llm.generate(['... is the capital of France', 'What is the formula for the area of a circle'])
print(output.generations)

[[Generation(text='\n\nParis.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the area of a circle is A = πr², where r is the radius of the circle.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [13]:
print(output.generations[0][0].text)



Paris.


In [14]:
output = llm.generate(['write an original tagline for a burger restaurant'] * 3)

In [16]:
for o in output.generations:
    print(o[0].text, end='')



"A Burger Above the Rest - At Our Place!"

"Burgers That Will Leave You Wanting More!"

"Take a Bite Into Our Juicy Burgers!"

# ChatModels: GPT-3.5-Turbo and GPT-4

In [17]:
from langchain.schema import(
    AIMessage,  #assistant role
    HumanMessage, #user role
    SystemMessage  #system role
)
from langchain.chat_models import ChatOpenAI

In [20]:
chat  = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)  #or gpt-3.5-turbo
messages = [
    SystemMessage(content='You are a physicist and respond only in french.'),
    HumanMessage(content='Explain quantum mechanic in one sentence')
]
output = chat(messages)

In [22]:
print(output.content)

La mécanique quantique est la théorie physique qui décrit le comportement des particules subatomiques et leurs interactions, basée sur le principe de superposition et la dualité onde-particule.


## Prompt templates

In [24]:
from langchain import PromptTemplate    

In [25]:
template = '''YOu are an experienced virologist.
Write a few sentences about the following {virus} in {language}'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)
print(prompt)

input_variables=['virus', 'language'] output_parser=None partial_variables={} template='YOu are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}' template_format='f-string' validate_template=True


In [26]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='ebola', language='french'))
print(output)



L’Ebola est une maladie virale grave qui se propage à travers les fluides corporels. Elle affecte principalement les humains et les primates, et peut entraîner des saignements internes et la mort. Les signes et symptômes comprennent la fièvre, la fatigue, la diarrhée et des maux de tête. La prévention et le traitement de l'Ebola reposent sur l'hygiène, la prise en charge médicale et l'isolement du patient.


## simple chains

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

template = '''YOu are an experienced virologist.
Write a few sentences about the following {virus} in {language}'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)

chain=LLMChain(llm=llm, prompt=prompt)

output=chain.run({'virus':'MSV', 'language':'french'})

In [29]:
print(output)

Le virus de la mosaïque du manioc (MSV) est un virus végétal qui affecte principalement les plantes de manioc. En tant que virologue expérimenté, j'ai étudié ce virus et ses effets sur les cultures agricoles. Le MSV provoque des symptômes de mosaïque sur les feuilles des plantes de manioc, ce qui réduit considérablement leur rendement et leur qualité. Des mesures de prévention et de contrôle sont nécessaires pour limiter la propagation de ce virus et protéger les cultures de manioc.


## Sequential chains

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 =OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Python programmer.
    Write a function that implements the concept of {concept}'''
)
chain1=LLMChain(llm=llm1, prompt=prompt1)

llm2 =OpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='''Given the python function {function} describe it as detailed as possible.'''
)
chain2=LLMChain(llm=llm2, prompt=prompt2)

overall_chain=SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('linear regression')



> Entering new  chain...
.

def linear_regression(x_values, y_values):
    """
    Returns the coefficients of a linear regression line
    given a list of x-values and y-values
    """
    x_mean = sum(x_values) / len(x_values)
    y_mean = sum(y_values) / len(y_values) 
    numerator = 0
    denominator = 0
    for i in range(len(x_values)):
        numerator += (x_values[i] - x_mean) * (y_values[i] - y_mean)
        denominator += (x_values[i] - x_mean) ** 2
    m = numerator / denominator
    b = y_mean - (m * x_mean)
    return m, b
The given python function `linear_regression` calculates the coefficients of a linear regression line based on a list of x-values and y-values.

The function starts by calculating the mean (average) of the x-values and y-values using the formulas:
- `x_mean = sum(x_values) / len(x_values)` to calculate the mean of x-values
- `y_mean = sum(y_values) / len(y_values)` to calculate the mean of y-values

Next, the function initializes two variables `numer

In [38]:
output = overall_chain.run('softmax')



> Entering new  chain...
 to input scores.

def softmax(x):
    """Compute softmax values for each sets of scores in x"""
    return np.exp(x) / np.sum(np.exp(x), axis=0)
The function "softmax" is implementing the softmax activation function in Python. This activation function is commonly used when dealing with multi-class classification problems. It takes a vector of scores as input and computes the softmax values for each set of scores.

The function calculates the exponential of each element in the input vector x using np.exp(x). This is because softmax requires exponentiation to make the scores positive and meaningful for probabilities. 

Then, it computes the sum of the exponentiated scores along the specified axis (axis=0 in this case) using np.sum(). This sum is necessary to normalize the scores and ensure they add up to 1.

Finally, it divides each exponentiated score by the sum computed earlier to obtain the softmax values for each score. This is done by dividing np.exp(x) b